In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().resolve()
if ROOT.name == "Note": ROOT = ROOT.parents[0]
DATA = ROOT / "DATA"
CHAN = ROOT / "chan_data"

BT_CFG = dict(
    symbols = ["BTCUSDT","ETHUSDT"],
    allow_tfs = ["5m","15m","30m","1h","4h"],
    allow_tags = ["1B","1S","2B","2S","3B","3S"],
    min_score_final = 1.6,
    throttle_minutes = 10,
    atr_lookback = 14,
    k_sl_atr = 1.5,
    k_tp_atr = 3.0,
    max_hold_bars = 60,
    capital = 10000.0,
)

def read_kline(sym, tf):
    p = DATA/sym/f"{tf}.csv"
    df = pd.read_csv(p)
    if "timestamp" not in df.columns:
        for c in ("time","datetime","ts"):
            if c in df.columns:
                df = df.rename(columns={c:"timestamp"}); break
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
    return df.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)

def read_signals(sym):
    p = CHAN/"signals"/"trading"/sym/"multi_tf_trading_signals.csv"
    df = pd.read_csv(p, parse_dates=["ts"])
    df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")
    return df

def atr_like(df, lookback=14):
    if {"high","low","close"}.issubset(df.columns):
        h,l,c_prev = df["high"].astype(float), df["low"].astype(float), df["close"].astype(float).shift(1)
        tr = pd.concat([(h-l).abs(), (h-c_prev).abs(), (l-c_prev).abs()], axis=1).max(axis=1)
        return tr.rolling(lookback, min_periods=1).mean()
    return df["close"].astype(float).diff().abs().rolling(lookback, min_periods=1).mean()

def simulate_trade_stream(signals_df, k_by_tf, cfg):
    trades = []
    if cfg["throttle_minutes"] > 0:
        signals_df = signals_df.sort_values(["ts","score_final"], ascending=[True, False])
        kept, last_ts_side = [], {}
        gap = pd.Timedelta(minutes=cfg["throttle_minutes"])
        for r in signals_df.itertuples():
            key = r.side
            t = pd.to_datetime(r.ts)
            if key not in last_ts_side or t - last_ts_side[key] >= gap:
                kept.append(r)
                last_ts_side[key] = t
        signals_df = pd.DataFrame([x._asdict() for x in kept])

    for r in signals_df.itertuples():
        tf = r.tf
        side = r.side
        ts = pd.to_datetime(r.ts)
        k = k_by_tf[tf]
        tss = pd.to_datetime(k["timestamp"])

        i = tss.searchsorted(ts)
        if i >= len(k)-1:  # 没有下一根
            continue
        entry_i = i + 1
        entry_ts = pd.to_datetime(k["timestamp"].iloc[entry_i])
        entry = float(k["open"].iloc[entry_i])

        atr_series = atr_like(k.iloc[:entry_i+1], cfg["atr_lookback"])
        atrv = float(atr_series.iloc[-1]) if not np.isnan(atr_series.iloc[-1]) else 0.0
        sl_k = max(1e-9, cfg["k_sl_atr"]) * max(1e-9, atrv)
        tp_k = max(1e-9, cfg["k_tp_atr"]) * max(1e-9, atrv)

        if side == "buy":
            sl = entry - sl_k
            tp = entry + tp_k
        else:
            sl = entry + sl_k
            tp = entry - tp_k

        exit_ts = exit_price = None
        end_i = min(len(k)-1, entry_i + cfg["max_hold_bars"])
        for j in range(entry_i, end_i+1):
            lo = float(k["low"].iloc[j]) if "low" in k.columns else float(k["close"].iloc[j])
            hi = float(k["high"].iloc[j]) if "high" in k.columns else float(k["close"].iloc[j])

            hit_sl = (lo <= sl) if side=="buy" else (hi >= sl)
            hit_tp = (hi >= tp) if side=="buy" else (lo <= tp)

            if hit_sl:
                exit_ts = pd.to_datetime(k["timestamp"].iloc[j])
                exit_price = sl
                break
            if hit_tp:
                exit_ts = pd.to_datetime(k["timestamp"].iloc[j])
                exit_price = tp
                break

        if exit_ts is None:
            exit_ts = pd.to_datetime(k["timestamp"].iloc[end_i])
            exit_price = float(k["close"].iloc[end_i])

        pnl = (exit_price - entry) if side=="buy" else (entry - exit_price)
        r_multiple = pnl / max(1e-9, sl_k)
        ret_pct = pnl / entry

        trades.append(dict(
            tf=tf, side=side, tag=r.tag, ts_signal=ts, ts_entry=entry_ts, ts_exit=exit_ts,
            entry=entry, exit=exit_price, sl=sl, tp=tp, atr=atrv,
            r=r_multiple, ret_pct=ret_pct, score_final=float(r.score_final) if "score_final" in r._fields else np.nan
        ))
    return pd.DataFrame(trades)

def backtest_symbol(sym, cfg=BT_CFG):
    sig = read_signals(sym)
    sig = sig[
        sig["tf"].isin(cfg["allow_tfs"])
        & sig["tag"].isin(cfg["allow_tags"])
        & (sig["score_final"] >= cfg["min_score_final"])
    ].sort_values("ts").reset_index(drop=True)

    k_by_tf = {tf: read_kline(sym, tf) for tf in cfg["allow_tfs"]}

    need_cols = {"timestamp","open","close"}
    for tf, k in k_by_tf.items():
        missing = need_cols - set(k.columns)
        if missing:
            raise ValueError(f"{sym}-{tf} 缺少必要列: {missing}")

    trades = simulate_trade_stream(sig, k_by_tf, cfg)

    out_dir = CHAN/"signals"/"backtest"/sym
    out_dir.mkdir(parents=True, exist_ok=True)
    trades.to_csv(out_dir/"trades.csv", index=False)

    if trades.empty:
        print(f"[BT] {sym}: no trades after filters.")
        return trades

    win = (trades["r"] > 0).mean()
    avg_r = trades["r"].mean()
    avg_win = trades.loc[trades["r"]>0, "r"].mean() if (trades["r"]>0).any() else np.nan
    avg_loss = trades.loc[trades["r"]<=0, "r"].mean() if (trades["r"]<=0).any() else np.nan
    max_dd = (trades["r"].cumsum().cummax() - trades["r"].cumsum()).max()
    print(f"[BT] {sym}: n={len(trades)}, win%={win:.2%}, avgR={avg_r:.2f}, avgW={avg_win:.2f}, avgL={avg_loss:.2f}, maxDD(R)={max_dd:.2f}")
    print(f" -> {out_dir/'trades.csv'}")

    g = trades.groupby(["tf","side","tag"]).size().sort_values(ascending=False).head(10)
    print("\n[top buckets]\n", g)
    return trades

all_trades = {}
for s in BT_CFG["symbols"]:
    all_trades[s] = backtest_symbol(s, BT_CFG)

all_df = pd.concat({k:v for k,v in all_trades.items() if v is not None and not v.empty}, names=["symbol"])
print("\n[ALL] head\n", all_df.head())


[BT] BTCUSDT: n=64, win%=73.44%, avgR=1.19, avgW=1.96, avgL=-0.95, maxDD(R)=4.00
 -> /Users/cc/PycharmProjects/ChanLab/chan_data/signals/backtest/BTCUSDT/trades.csv

[top buckets]
 tf   side  tag
15m  buy   1B     10
30m  sell  1S     10
15m  sell  1S      9
30m  buy   1B      7
5m   buy   1B      5
1h   buy   1B      4
     sell  1S      4
30m  buy   3B      3
5m   sell  1S      3
1h   sell  3S      2
dtype: int64
[BT] ETHUSDT: n=117, win%=69.23%, avgR=1.04, avgW=1.94, avgL=-1.00, maxDD(R)=4.00
 -> /Users/cc/PycharmProjects/ChanLab/chan_data/signals/backtest/ETHUSDT/trades.csv

[top buckets]
 tf   side  tag
5m   sell  1S     26
15m  sell  1S     17
5m   buy   1B     13
15m  buy   1B      9
30m  buy   1B      9
     sell  1S      7
1h   sell  1S      5
30m  buy   3B      4
1h   buy   3B      4
15m  buy   3B      3
dtype: int64

[ALL] head
             tf  side tag                 ts_signal                  ts_entry  \
symbol                                                              